In [12]:
import pandas as pd
from arcgis import GIS
gis = GIS()
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import requests
import arcpy

import geopandas as gpd

In [13]:
#strRelease = "WF TDM v9.0.1"
lstFieldsFromForecasts = ['SEGID','MF2023','MF2028','MF2032','MF2042','MF2050','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','ADJHIST','DONOTUSE']

In [14]:
# Your initial DataFrame
dfSegmentFilenames = pd.DataFrame([
    ["CO_FIPS==57 & SUBAREAID==0"                         , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ], # remaining Weber
    ["CO_FIPS==3 & SUBAREAID==0"                          , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ], # remaining Box Elder
    ["F_AREA=='Iron' | F_AREA=='Cache' | F_AREA=='Dixie'" , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ], # Iron, Cache, Dixie Final
    ["PLANAREA=='WFRC'"                                   , 'data/segments/WFv901_Segments_20240409_GeoKeys_AADTs_Factors.shp', False], # WFRC final
    ["PLANAREA=='MAG'"                                    , 'data/segments/WF_Segments_20240326_Draft.shp'                    , False], # MAG final
    ["CO_FIPS==43 & SUBAREAID==4"                         , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ], # Summit County
    ["CO_FIPS==51 & SUBAREAID==4"                         , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ], # Wasatch County
 ], columns=['filter', 'filename', 'updateRoute'])

# Initialize an empty list to store each filtered GeoDataFrame
_filteredGdfs = []

# Iterate through each row in your DataFrame
for _, row in dfSegmentFilenames.iterrows():
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(row['filename'])
    
    # Use the `query` method to apply the filter from the 'filter' column
    _filteredGdf = gdf.query(row['filter'])
    
    if row['updateRoute']:
        # Assuming 'SEGID' is the correct column to apply this operation
        _filteredGdf.loc[:, 'ROUTE'] = _filteredGdf['SEGID'].apply(lambda x: x.split('_')[0] if '_' in x else x)

    print("Filter:" + row['filter'] + " -- Segments: " + str(_filteredGdf.shape[0]))

    # Add the filtered GeoDataFrame to the list
    _filteredGdfs.append(_filteredGdf)

# Concatenate all filtered GeoDataFrames into a single GeoDataFrame
gdfs = pd.concat(_filteredGdfs, ignore_index=True)

# Assuming you want to display or further process `gdfs`
display(gdfs)


C:\Users\bhereth\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Filter:CO_FIPS==57 & SUBAREAID==0 -- Segments: 36
Filter:CO_FIPS==3 & SUBAREAID==0 -- Segments: 196
Filter:F_AREA=='Iron' | F_AREA=='Cache' | F_AREA=='Dixie' -- Segments: 1353
Filter:PLANAREA=='WFRC' -- Segments: 3252
Filter:PLANAREA=='MAG' -- Segments: 2090
Filter:CO_FIPS==43 & SUBAREAID==4 -- Segments: 205
Filter:CO_FIPS==51 & SUBAREAID==4 -- Segments: 174


,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0,0039_010.7,10.679,13.792,3.115424,UDOT,9056.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0039_013.8,13.792,16.561,2.774178,UDOT,4185.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0039_016.6,16.561,17.492,0.932776,UDOT,3950.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0039_017.5,17.492,19.324,1.834455,UDOT,6903.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0039_019.3,19.324,21.439,2.123916,UDOT,2438.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,0,UDOT_7020,0.000,0.000,1.067629,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7302,0,UDOT_7030,0.000,0.000,3.466911,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7303,0,UDOT_7041,0.000,0.000,0.137915,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7304,0,UDOT_7042,0.000,0.000,0.080061,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
gdfs[gdfs['SEGID']=='2610_017.8']

,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
7100,0,2610_017.8,17.829,23.004,5.191988,UDOT,192.0,4.0,43.0,Summit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# add observed
dfObsVolSource = pd.read_csv('intermediate/obsvolsource.csv')
display(dfObsVolSource)
dfAadt = pd.read_csv('intermediate/aadt.csv')

dfGetObsAadt = pd.merge(dfObsVolSource,dfAadt,on='SOURCE')
display(dfGetObsAadt)

dfGetObsAadt = dfGetObsAadt[['SEGID','YEAR','AADT']]

dfGetObsAadt['YEAR'] = 'AADT' + dfGetObsAadt['YEAR'].astype(str)

dfGetObsAadt = dfGetObsAadt.drop_duplicates(subset=['SEGID','YEAR'])

dfGetObsAadt = dfGetObsAadt.pivot(index='SEGID',columns='YEAR', values='AADT').fillna(0).reset_index()

display(dfGetObsAadt)

,SOURCE,SUBAREAID
0,AADTHistory.xlsx,0
1,AADTHistory.xlsx,2
2,AADTHistory.xlsx,3
3,AADTHistory.xlsx,4
4,AADTHistory.xlsx,5
5,WF_Segments_20240326_Draft.shp,1


,SOURCE,SUBAREAID,SEGID,YEAR,AADT
0,AADTHistory.xlsx,0,0065_000.1,1981,530
1,AADTHistory.xlsx,0,0065_000.1,1982,545
2,AADTHistory.xlsx,0,0065_000.1,1983,495
3,AADTHistory.xlsx,0,0065_000.1,1984,555
4,AADTHistory.xlsx,0,0065_000.1,1985,575
...,...,...,...,...,...
702380,WF_Segments_20240326_Draft.shp,1,MAG_6604,2018,500
702381,WF_Segments_20240326_Draft.shp,1,MAG_6604,2019,500
702382,WF_Segments_20240326_Draft.shp,1,MAG_6604,2020,500
702383,WF_Segments_20240326_Draft.shp,1,MAG_6604,2021,500


YEAR,SEGID,AADT1981,AADT1982,AADT1983,AADT1984,AADT1985,AADT1986,AADT1987,AADT1988,AADT1989,...,AADT2013,AADT2014,AADT2015,AADT2016,AADT2017,AADT2018,AADT2019,AADT2020,AADT2021,AADT2022
0,0006_000.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
1,0006_000.7,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
2,0006_016.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,330.0,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0
3,0006_046.0,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0
4,0006_060.2,325.0,335.0,430.0,580.0,585.0,585.0,595.0,0.0,400.0,...,340.0,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7281,WFRC_8010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1000.0,1053.0,1081.0,1092.0,1112.0,993.0,1076.0,1087.0
7282,WFRC_8220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7555.0,7635.0,15375.0,16191.0,16628.0,16794.0,17096.0,15267.0,16549.0,16715.0
7283,WFRC_8262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13055.0,13200.0,13770.0,14498.0,6489.0,6554.0,6672.0,5958.0,6458.0,6523.0
7284,WFRC_8419,15075.0,15150.0,15150.0,15440.0,15850.0,16010.0,16170.0,16250.0,15430.0,...,15630.0,15800.0,16480.0,17353.0,17822.0,18000.0,18324.0,16363.0,17737.0,17915.0


In [17]:
# Assuming gdfs and dfGetObsAadt are already defined and loaded
# Step 1: Merge the dataframes on 'SEGID'
merged_df = pd.merge(gdfs, dfGetObsAadt, on='SEGID', suffixes=('', '_new'))

# Step 2: Update the columns in gdfs with those from dfGetObsAadt where they exist
# Perform an inner merge to get only the rows with matching SEGIDs
common_columns = [col for col in dfGetObsAadt.columns if col in gdfs.columns and col != 'SEGID']
merged_common = gdfs.merge(merged_df, on='SEGID', how='inner')

# Update only the matching rows in gdfs
for col in common_columns:
    gdfs.loc[gdfs['SEGID'].isin(merged_common['SEGID']), col] = merged_common[col + '_new'].values

display(gdfs)

,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0,0039_010.7,10.679,13.792,3.115424,UDOT,9056.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0039_013.8,13.792,16.561,2.774178,UDOT,4185.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0039_016.6,16.561,17.492,0.932776,UDOT,3950.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0039_017.5,17.492,19.324,1.834455,UDOT,6903.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0039_019.3,19.324,21.439,2.123916,UDOT,2438.0,0.0,57.0,UDOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,0,UDOT_7020,0.000,0.000,1.067629,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7302,0,UDOT_7030,0.000,0.000,3.466911,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7303,0,UDOT_7041,0.000,0.000,0.137915,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7304,0,UDOT_7042,0.000,0.000,0.080061,UDOT,0.0,4.0,51.0,MAG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# double check that the above merge worked by looking at a segment
gdfs[gdfs['SEGID']=='2610_017.8'][['SEGID','AADT2017']]

,SEGID,AADT2017
7100,2610_017.8,187.0


In [19]:
# drop unneeded columns and fill nas
sdfSegments = GeoAccessor.from_geodataframe(gdfs)
sdfSegments.drop(columns=['Id','F_AREA'], inplace=True) # only used for art of forecasting tool
sdfSegments

,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,CCSGROUP19,FACMANADJ,...,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE
0,0039_010.7,10.679,13.792,3.115424,UDOT,9056.0,0.0,57.0,COQ,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[429484.3499999996, 4567268.140000..."
1,0039_013.8,13.792,16.561,2.774178,UDOT,4185.0,0.0,57.0,COQ,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[433479.0800000001, 4566770.890000..."
2,0039_016.6,16.561,17.492,0.932776,UDOT,3950.0,0.0,57.0,COQ,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[434947.6900000004, 4566576.4], [4..."
3,0039_017.5,17.492,19.324,1.834455,UDOT,6903.0,0.0,57.0,COQ,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[436208.5, 4568166.199999999], [43..."
4,0039_019.3,19.324,21.439,2.123916,UDOT,2438.0,0.0,57.0,COP,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[439617.8099999996, 4567982.91], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,0.000,0.000,1.067629,UDOT,0.0,4.0,51.0,COT,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[466893.2999999998, 4484155.0], [4..."
7302,UDOT_7030,0.000,0.000,3.466911,UDOT,0.0,4.0,51.0,COP,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[462056.8099999996, 4498762.58], [..."
7303,UDOT_7041,0.000,0.000,0.137915,UDOT,0.0,4.0,51.0,COT,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[465682.3700000001, 4480915.51], [..."
7304,UDOT_7042,0.000,0.000,0.080061,UDOT,0.0,4.0,51.0,COU,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[465685.5700000003, 4483225.970000..."


In [20]:
sdfSegments.groupby(['PLANAREA']).agg(COUNT=('SEGID','count'))

,COUNT
PLANAREA,
Cache,402
Dixie,555
MAG,2090
UDOT,1007
WFRC,3252


In [21]:
# Connect to GIS
gis = GIS()

# Access the feature layer
feature_layer_url = 'https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0'
feature_layer = FeatureLayer(feature_layer_url)

# Convert to Spatially Enabled DataFrame
sdfForecasts = feature_layer.query().sdf
sdfForecasts = sdfForecasts[lstFieldsFromForecasts] # filter columns by list

# Display the first few rows of the dataframe
display(sdfForecasts)


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,DONOTUSE
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0,0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [22]:
sdfForecasts[sdfForecasts['SEGID']=='2610_017.8']

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,DONOTUSE
8486,2610_017.8,200.0,200.0,200.0,400.0,400.0,50.0,50.0,100.0,-50.0,0.0,0.0,1


In [23]:
years = [2023, 2028, 2032, 2042, 2050]

for year in years:
    # F  : Forecast 
    # MF : Model Forecast (model volume + base year adjustment)
    # ADJ: Manual Adjustment
    sdfForecasts[f'F{year}'] = sdfForecasts[f'MF{year}'] + sdfForecasts[f'ADJ{year}'] + sdfForecasts['ADJHIST']

sdfForecasts

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,DONOTUSE,F2023,F2028,F2032,F2042,F2050
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0,0,20000.0,21000.0,21500.0,21500.0,22000.0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20000.0,21000.0,21000.0,22000.0,23000.0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0,0,16000.0,17000.0,17000.0,18500.0,19500.0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17000.0,18000.0,18000.0,19500.0,20000.0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17000.0,17500.0,17500.0,18500.0,19000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0


In [24]:
sdfForecasts.columns

Index(['SEGID', 'MF2023', 'MF2028', 'MF2032', 'MF2042', 'MF2050', 'ADJ2023',
       'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050', 'ADJHIST', 'DONOTUSE',
       'F2023', 'F2028', 'F2032', 'F2042', 'F2050'],
      dtype='object')

In [25]:
display(sdfSegments.columns)

# Your initial list of fields you consider dropping, excluding 'SEGID'
columns_to_drop = [column for column in lstFieldsFromForecasts if column != 'SEGID']

# Calculate the intersection of columns to drop and existing columns
columns_to_drop_exist = [column for column in columns_to_drop if column in sdfSegments.columns]

# Drop these columns from sdfSegments if they exist
sdfSegments.drop(columns=columns_to_drop_exist, inplace=True)

# Display remaining columns in sdfSegments to verify
display(sdfSegments.columns)


Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'F2032', 'F2042', 'F2050', 'CH19TO50', 'CH19TO23', 'CH23TO28',
       'CH28TO32', 'CH32TO42', 'CH42TO50', 'SHAPE'],
      dtype='object', length=122)

Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'F2032', 'F2042', 'F2050', 'CH19TO50', 'CH19TO23', 'CH23TO28',
       'CH28TO32', 'CH32TO42', 'CH42TO50', 'SHAPE'],
      dtype='object', length=122)

In [26]:
sdfForecastsPlusSegments = pd.merge(sdfForecasts,sdfSegments.drop(columns=['F2019','F2023','F2028','F2032','F2042','F2050']),on='SEGID',how='right')
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,CUTRK2011,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE
0,0039_010.7,9200.0,10500.0,10500.0,12500.0,14000.0,0.0,-1000.0,-800.0,-2500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(429484.3499999996, 4567268.140000..."
1,0039_013.8,4500.0,5100.0,5400.0,6200.0,7100.0,0.0,-200.0,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(433479.0800000001, 4566770.890000..."
2,0039_016.6,4200.0,4800.0,5000.0,5700.0,6300.0,0.0,-300.0,-300.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(434947.6900000004, 4566576.4), (4..."
3,0039_017.5,7600.0,8100.0,8200.0,9000.0,10000.0,0.0,300.0,500.0,1500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(436208.5, 4568166.199999999), (43..."
4,0039_019.3,2800.0,2900.0,3000.0,3400.0,3500.0,-300.0,-200.0,-200.0,-200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(439617.8099999996, 4567982.91), (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,0.0,450.0,450.0,500.0,450.0,0.0,0.0,0.0,-50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(466893.2999999998, 4484155.0), (4..."
7302,UDOT_7030,750.0,1200.0,1400.0,2000.0,2500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(462056.8099999996, 4498762.58), (..."
7303,UDOT_7041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(465682.3700000001, 4480915.51), (..."
7304,UDOT_7042,0.0,0.0,2000.0,2300.0,2600.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(465685.5700000003, 4483225.970000..."


In [27]:
# cacluate and adjusted base year
sdfForecastsPlusSegments['F2019'] = sdfForecastsPlusSegments['AADT2019'] + sdfForecastsPlusSegments['ADJHIST']

In [28]:
# calculate change fields
periods = [
    ('19TO50', 'F2019', 'F2050'),
    ('19TO23', 'F2019', 'F2023'),
    ('23TO28', 'F2023', 'F2028'),
    ('28TO32', 'F2028', 'F2032'),
    ('32TO42', 'F2032', 'F2042'),
    ('42TO50', 'F2042', 'F2050')
]

for period, start_year_key, end_year_key in periods:
    ch_key = f'CH{period}'
    sdfForecastsPlusSegments[ch_key] = sdfForecastsPlusSegments[end_year_key] - sdfForecastsPlusSegments[start_year_key]

sdfForecastsPlusSegments


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0039_010.7,9200.0,10500.0,10500.0,12500.0,14000.0,0.0,-1000.0,-800.0,-2500.0,...,NaN,NaN,1444.0,144.0,300.0,200.0,300.0,500.0,"{'paths': [[(429484.3499999996, 4567268.140000...",9056.0
1,0039_013.8,4500.0,5100.0,5400.0,6200.0,7100.0,0.0,-200.0,0.0,200.0,...,NaN,NaN,2915.0,315.0,400.0,500.0,1000.0,700.0,"{'paths': [[(433479.0800000001, 4566770.890000...",4185.0
2,0039_016.6,4200.0,4800.0,5000.0,5700.0,6300.0,0.0,-300.0,-300.0,0.0,...,NaN,NaN,2350.0,250.0,300.0,200.0,1000.0,600.0,"{'paths': [[(434947.6900000004, 4566576.4), (4...",3950.0
3,0039_017.5,7600.0,8100.0,8200.0,9000.0,10000.0,0.0,300.0,500.0,1500.0,...,NaN,NaN,5097.0,697.0,800.0,300.0,1800.0,1500.0,"{'paths': [[(436208.5, 4568166.199999999), (43...",6903.0
4,0039_019.3,2800.0,2900.0,3000.0,3400.0,3500.0,-300.0,-200.0,-200.0,-200.0,...,NaN,NaN,1062.0,62.0,200.0,100.0,400.0,300.0,"{'paths': [[(439617.8099999996, 4567982.91), (...",2438.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,0.0,450.0,450.0,500.0,450.0,0.0,0.0,0.0,-50.0,...,NaN,NaN,450.0,0.0,450.0,0.0,0.0,0.0,"{'paths': [[(466893.2999999998, 4484155.0), (4...",0.0
7302,UDOT_7030,750.0,1200.0,1400.0,2000.0,2500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,2500.0,750.0,450.0,200.0,600.0,500.0,"{'paths': [[(462056.8099999996, 4498762.58), (...",0.0
7303,UDOT_7041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(465682.3700000001, 4480915.51), (...",0.0
7304,UDOT_7042,0.0,0.0,2000.0,2300.0,2600.0,0.0,0.0,0.0,0.0,...,NaN,NaN,2600.0,0.0,0.0,2000.0,300.0,300.0,"{'paths': [[(465685.5700000003, 4483225.970000...",0.0


In [29]:
# drop do no use
sdfForecastsPlusSegments = sdfForecastsPlusSegments[sdfForecastsPlusSegments['DONOTUSE']==0]
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0039_010.7,9200.0,10500.0,10500.0,12500.0,14000.0,0.0,-1000.0,-800.0,-2500.0,...,NaN,NaN,1444.0,144.0,300.0,200.0,300.0,500.0,"{'paths': [[(429484.3499999996, 4567268.140000...",9056.0
1,0039_013.8,4500.0,5100.0,5400.0,6200.0,7100.0,0.0,-200.0,0.0,200.0,...,NaN,NaN,2915.0,315.0,400.0,500.0,1000.0,700.0,"{'paths': [[(433479.0800000001, 4566770.890000...",4185.0
2,0039_016.6,4200.0,4800.0,5000.0,5700.0,6300.0,0.0,-300.0,-300.0,0.0,...,NaN,NaN,2350.0,250.0,300.0,200.0,1000.0,600.0,"{'paths': [[(434947.6900000004, 4566576.4), (4...",3950.0
3,0039_017.5,7600.0,8100.0,8200.0,9000.0,10000.0,0.0,300.0,500.0,1500.0,...,NaN,NaN,5097.0,697.0,800.0,300.0,1800.0,1500.0,"{'paths': [[(436208.5, 4568166.199999999), (43...",6903.0
4,0039_019.3,2800.0,2900.0,3000.0,3400.0,3500.0,-300.0,-200.0,-200.0,-200.0,...,NaN,NaN,1062.0,62.0,200.0,100.0,400.0,300.0,"{'paths': [[(439617.8099999996, 4567982.91), (...",2438.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,0.0,450.0,450.0,500.0,450.0,0.0,0.0,0.0,-50.0,...,NaN,NaN,450.0,0.0,450.0,0.0,0.0,0.0,"{'paths': [[(466893.2999999998, 4484155.0), (4...",0.0
7302,UDOT_7030,750.0,1200.0,1400.0,2000.0,2500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,2500.0,750.0,450.0,200.0,600.0,500.0,"{'paths': [[(462056.8099999996, 4498762.58), (...",0.0
7303,UDOT_7041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(465682.3700000001, 4480915.51), (...",0.0
7304,UDOT_7042,0.0,0.0,2000.0,2300.0,2600.0,0.0,0.0,0.0,0.0,...,NaN,NaN,2600.0,0.0,0.0,2000.0,300.0,300.0,"{'paths': [[(465685.5700000003, 4483225.970000...",0.0


In [30]:
lstDropFields = [item for item in lstFieldsFromForecasts if item != 'SEGID']
sdfForecastsPlusSegments.drop(columns=lstDropFields, inplace=True)
sdfForecastsPlusSegments

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,SEGID,F2023,F2028,F2032,F2042,F2050,BMP,EMP,DISTANCE,PLANAREA,...,SUTRK2010,CUTRK2010,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,SHAPE,F2019
0,0039_010.7,9200.0,9500.0,9700.0,10000.0,10500.0,10.679,13.792,3.115424,UDOT,...,NaN,NaN,1444.0,144.0,300.0,200.0,300.0,500.0,"{'paths': [[(429484.3499999996, 4567268.140000...",9056.0
1,0039_013.8,4500.0,4900.0,5400.0,6400.0,7100.0,13.792,16.561,2.774178,UDOT,...,NaN,NaN,2915.0,315.0,400.0,500.0,1000.0,700.0,"{'paths': [[(433479.0800000001, 4566770.890000...",4185.0
2,0039_016.6,4200.0,4500.0,4700.0,5700.0,6300.0,16.561,17.492,0.932776,UDOT,...,NaN,NaN,2350.0,250.0,300.0,200.0,1000.0,600.0,"{'paths': [[(434947.6900000004, 4566576.4), (4...",3950.0
3,0039_017.5,7600.0,8400.0,8700.0,10500.0,12000.0,17.492,19.324,1.834455,UDOT,...,NaN,NaN,5097.0,697.0,800.0,300.0,1800.0,1500.0,"{'paths': [[(436208.5, 4568166.199999999), (43...",6903.0
4,0039_019.3,2500.0,2700.0,2800.0,3200.0,3500.0,19.324,21.439,2.123916,UDOT,...,NaN,NaN,1062.0,62.0,200.0,100.0,400.0,300.0,"{'paths': [[(439617.8099999996, 4567982.91), (...",2438.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,0.0,450.0,450.0,450.0,450.0,0.000,0.000,1.067629,UDOT,...,NaN,NaN,450.0,0.0,450.0,0.0,0.0,0.0,"{'paths': [[(466893.2999999998, 4484155.0), (4...",0.0
7302,UDOT_7030,750.0,1200.0,1400.0,2000.0,2500.0,0.000,0.000,3.466911,UDOT,...,NaN,NaN,2500.0,750.0,450.0,200.0,600.0,500.0,"{'paths': [[(462056.8099999996, 4498762.58), (...",0.0
7303,UDOT_7041,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.137915,UDOT,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,"{'paths': [[(465682.3700000001, 4480915.51), (...",0.0
7304,UDOT_7042,0.0,0.0,2000.0,2300.0,2600.0,0.000,0.000,0.080061,UDOT,...,NaN,NaN,2600.0,0.0,0.0,2000.0,300.0,300.0,"{'paths': [[(465685.5700000003, 4483225.970000...",0.0


In [31]:
sdfForecastsPlusSegments.columns


# Set display options to ensure no truncation
pd.set_option('display.max_columns', None)  # Ensures all columns are shown
pd.set_option('display.max_seq_items', None)  # For potentially long sequences

# Now, when you print or display the DataFrame, all column names will be shown
print(sdfForecastsPlusSegments.columns)

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_seq_items')


Index(['SEGID', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050', 'BMP', 'EMP',
       'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID', 'CO_FIPS',
       'CCSGROUP19', 'FACMANADJ', 'FAC_APR', 'FAC_AUG', 'FAC_DEC', 'FAC_FAL',
       'FAC_FEB', 'FAC_FRI', 'FAC_JAN', 'FAC_JUL', 'FAC_JUN', 'FAC_MAR',
       'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY', 'FAC_MON', 'FAC_NOV', 'FAC_OCT',
       'FAC_SAT', 'FAC_SEP', 'FAC_SPR', 'FAC_SUM', 'FAC_SUN', 'FAC_THU',
       'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WED', 'FAC_WEMAX', 'FAC_WIN',
       'ROUTE', 'Id_x', 'Id_y', 'AADTSTN', 'AADT2022', 'AADT2021', 'AADT2020',
       'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013',
       'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007',
       'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001',
       'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995',
       'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989',
      

In [32]:
lstFinalCols = ['SEGID',
                'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'SUBAREAID', 'PLANAREA',
                'F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050',
                'CH19TO50', 'CH19TO23', 'CH23TO28', 'CH28TO32', 'CH32TO42', 'CH42TO50', 
                'AADTSTN',
                'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017',
                'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011',
                'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005',
                'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999',
                'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993',
                'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987',
                'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981',
                'SUTRK2022', 'CUTRK2022', 'SUTRK2021', 'CUTRK2021', 'SUTRK2020',
                'CUTRK2020', 'SUTRK2019', 'CUTRK2019', 'SUTRK2018', 'CUTRK2018',
                'SUTRK2017', 'CUTRK2017', 'SUTRK2016', 'CUTRK2016', 'SUTRK2015',
                'CUTRK2015', 'SUTRK2014', 'CUTRK2014', 'SUTRK2013', 'CUTRK2013',
                'SUTRK2012', 'CUTRK2012', 'SUTRK2011', 'CUTRK2011', 'SUTRK2010',
                'CUTRK2010', 'CCSGROUP19',
                'FAC_MON', 'FAC_TUE', 'FAC_WED', 'FAC_THU', 'FAC_FRI', 'FAC_SAT', 'FAC_SUN',
                'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WEMAX',
                'FAC_SPR', 'FAC_SUM', 'FAC_FAL', 'FAC_WIN', 
                'FAC_JAN', 'FAC_FEB', 'FAC_MAR', 'FAC_APR', 'FAC_MAY', 'FAC_JUN',
                'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
                'FAC_MAXMO','FAC_MAX',
                'FACMANADJ', 'SHAPE']


sdfForecastsPlusSegmentsFinal = sdfForecastsPlusSegments[lstFinalCols].copy()

# For numeric columns
numeric_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['int', 'float']).columns
sdfForecastsPlusSegmentsFinal[numeric_cols] = sdfForecastsPlusSegmentsFinal[numeric_cols].fillna(0)

# For string/object columns
string_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['object']).columns
sdfForecastsPlusSegmentsFinal[string_cols] = sdfForecastsPlusSegmentsFinal[string_cols].fillna('')

# If you also have categorical columns and want to treat them as strings:
categorical_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['category']).columns
if not categorical_cols.empty:  # Check if there are any categorical columns
    sdfForecastsPlusSegmentsFinal[categorical_cols] = sdfForecastsPlusSegmentsFinal[categorical_cols].fillna('')

sdfForecastsPlusSegmentsFinal

,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0039_010.7,0039,10.679,13.792,3.115424,57.0,0.0,UDOT,9056.0,9200.0,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7,1.2056,0.0,"{'paths': [[(429484.3499999996, 4567268.140000..."
1,0039_013.8,0039,13.792,16.561,2.774178,57.0,0.0,UDOT,4185.0,4500.0,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7,1.2056,0.0,"{'paths': [[(433479.0800000001, 4566770.890000..."
2,0039_016.6,0039,16.561,17.492,0.932776,57.0,0.0,UDOT,3950.0,4200.0,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7,1.2056,0.0,"{'paths': [[(434947.6900000004, 4566576.4), (4..."
3,0039_017.5,0039,17.492,19.324,1.834455,57.0,0.0,UDOT,6903.0,7600.0,...,1.2056,1.1437,1.0572,1.0385,0.9133,0.8849,7,1.2056,0.0,"{'paths': [[(436208.5, 4568166.199999999), (43..."
4,0039_019.3,0039,19.324,21.439,2.123916,57.0,0.0,UDOT,2438.0,2500.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[(439617.8099999996, 4567982.91), (..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,UDOT_7020,UDOT,0.000,0.000,1.067629,51.0,4.0,UDOT,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[(466893.2999999998, 4484155.0), (4..."
7302,UDOT_7030,UDOT,0.000,0.000,3.466911,51.0,4.0,UDOT,0.0,750.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[(462056.8099999996, 4498762.58), (..."
7303,UDOT_7041,UDOT,0.000,0.000,0.137915,51.0,4.0,UDOT,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[(465682.3700000001, 4480915.51), (..."
7304,UDOT_7042,UDOT,0.000,0.000,0.080061,51.0,4.0,UDOT,0.0,0.0,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{'paths': [[(465685.5700000003, 4483225.970000..."


In [33]:
sdfForecastsPlusSegmentsFinal.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\Traffic-Volume-Historic-and-Forecast.shp'